# Project

## Download Data

In [1]:
import urllib
import json
import pandas as pd
from bd_addons.interpro_data import *
from bd_addons.HmmPy import *
from bd_addons.metrics import *
from Bio import SearchIO, SeqIO

url = "https://www.ebi.ac.uk/interpro/api/protein/reviewed/entry/pfam/pf03060?format=json"

In [2]:
metadata, entries, gt = get_data(url, 1)


# 8 - Evaluate Ability of Matching sequences
Once we made the PSI-BLAST e HMM-searches we will have some files where all the sequences are... we should:

- Find a format that we can parse (there should be options in the documentations of thevarious methods);
- Decide a threshold for the E-value and take all the hits that are below that E value;
- All the remaining sequences (call them OUT_SEQs)are the ones that, according to our model, belong to the Domain family.
- Make a Confusion Matrix comparing OUT_SEQs with the ground truth.
- compute accuracy, sensitivity, specificty etc

## HMM Searches Performance

In [3]:
parsed_hmm_t_C_df, parsed_hmm_t_M_df, parsed_hmm_t_O_df = parse_tblouts()

The name file or folder is incorrect.                  An error has occurred.


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\manen\\Desktop\\BD_Project\\code\\BD_Project_DS\\data_team_1\\HMMs\\HMM_M\\hmmsearch_out_M_1_swissprot.tblout'

In [29]:
gt_acc = gt.accession.to_list()
swiss_prot = 563972
parsed_hmm_t_C_df['in_gt'] = parsed_hmm_t_C_df['target name'].apply(lambda x: split('-', x.replace('|', '-'))[1]).apply(lambda x: 1 if x in gt_acc else 0)
# print(parsed_hmm_t_C_df['in_gt'])
# parsed_hmm_t_C_df['in_gt'] = parsed_hmm_t_C_df['in_gt'].apply(lambda x: 1 if x in gt_acc else 0)
accuracy = (parsed_hmm_t_C_df['in_gt'].sum() + swiss_prot - len(parsed_hmm_t_C_df['in_gt'])) / (swiss_prot)
precision = parsed_hmm_t_C_df['in_gt'].sum() / len(parsed_hmm_t_C_df['in_gt'])
# recall = parsed_hmm_t_C_df['in_gt'].sum() / (parsed_hmm_t_C_df['in_gt'].sum() + positive in swiss_prot - hmm search)

print('accuracy', accuracy, 'precision', precision)

accuracy 0.9962303093061358 precision 0.010702652396463471


In [4]:
parsed_hmm_domt_C_df, parsed_hmm_domt_M_df, parsed_hmm_domt_O_df = parse_domtblouts()

## PSSM Model Performance (C1)

In [4]:
iterations = 2
xml_path = ".\data_team_1\PSSMs\PSSM_C\out_psiblast_C_1_swissprot_{}iterations.xml".format(iterations)
print(xml_path)

.\data_team_1\PSSMs\PSSM_C\out_psiblast_C_1_swissprot_2iterations.xml


In [5]:
hits_dict = {}
blast_records = SearchIO.parse(xml_path, 'blast-xml')
for blast_record in blast_records:
    for rec in blast_record.hits:
        # print(rec.hsps[0].bitscore)
        hits_dict.setdefault("ids",[]).append(rec.id.split('|')[1])
        hits_dict.setdefault("e_value",[]).append(rec.hsps[0].evalue)
        hits_dict.setdefault("hit_start",[]).append(rec.hsps[0].hit_range[0])
        hits_dict.setdefault("hit_end",[]).append(rec.hsps[0].hit_range[1])
        hits_dict.setdefault("bitscore",[]).append(rec.hsps[0].bitscore)
        # hits_list.append((rec.id.split('|')[1], rec.hsps[0].evalue, rec.hsps[0].hit_range))

In [6]:
# per alessà: l'ho parsato io alla fine ciao stupidoh
# Da fare ora: valutare le varie metriche come hai già fatto con hmm

In [7]:
hits_df = pd.DataFrame.from_dict(hits_dict)
print(hits_df.shape)
hits_df.head()

(624, 5)


,ids,e_value,hit_start,hit_end,bitscore
0,Q6GB05,0.0,0,355,533.181
1,Q6GIG7,0.0,0,355,532.796
2,Q99VF6,0.0,0,355,532.796
3,A5IR97,0.0,0,355,532.796
4,A6U025,0.0,0,355,532.796


## PSSM Model Performance (C1_denoised1)

In [8]:
iterations = 2
xml_path = ".\data_team_1\PSSMs\PSSM_C\out_psiblast_C_1_denoised1_swissprot_{}iterations.xml".format(iterations)
print(xml_path)
hits_dict = {}
blast_records = SearchIO.parse(xml_path, 'blast-xml')
for blast_record in blast_records:
    for rec in blast_record.hits:
        # print(rec.hsps[0].bitscore)
        hits_dict.setdefault("ids",[]).append(rec.id.split('|')[1])
        hits_dict.setdefault("e_value",[]).append(rec.hsps[0].evalue)
        hits_dict.setdefault("hit_start",[]).append(rec.hsps[0].hit_range[0])
        hits_dict.setdefault("hit_end",[]).append(rec.hsps[0].hit_range[1])
        hits_dict.setdefault("bitscore",[]).append(rec.hsps[0].bitscore)
        # hits_list.append((rec.id.split('|')[1], rec.hsps[0].evalue, rec.hsps[0].hit_range))
   # print(blast_record.hits)

.\data_team_1\PSSMs\PSSM_C\out_psiblast_C_1_denoised1_swissprot_2iterations.xml


In [9]:
hits_df_denoised1 = pd.DataFrame.from_dict(hits_dict)
print(hits_df_denoised1.shape)
hits_df_denoised1.head(10)

(657, 5)


,ids,e_value,hit_start,hit_end,bitscore
0,Q6GIG7,0.0,8,348,513.124
1,Q6GB05,0.0,8,348,512.739
2,Q99VF6,0.0,8,348,512.739
3,A5IR97,0.0,8,348,512.739
4,A6U025,0.0,8,348,512.739
5,Q8NXG7,0.0,8,348,511.969
6,A8Z1H7,0.0,8,348,508.887
7,A6QFD2,0.0,8,348,508.887
8,Q5HHG4,0.0,8,348,508.887
9,Q2FZX9,0.0,8,348,508.887


# 9 - Evualuate the ability of matching domain positions